# Loading The Dependencies

In [3]:
import tensorflow as tf
import numpy as np

# Loading Data File

In [4]:
import helper

source_path='data/small_vocab_en'
target_path='data/small_vocab_fr'

source_text=helper.load_data(source_path)
target_text=helper.load_data(target_path)

## Exploring Data

In [5]:
print ("Data Set Information")
print ("The Number of Unique Words: %s"%(len({word: None for word in source_text.split()})))

sentence_arr=[]
sentences=source_text.split('\n')
for sentence in sentences:
    sentence_arr.append(sentences)
print ("The Number of Sentences are: %r "%(len(sentence_arr)))


#print ("The Number of Sentences are: %s"%(len(sentence.split()) for sentence in sentences))

#print (word_count)
#print 'The Number of Unique Words %r'(len(word: None for word  )

Data Set Information
The Number of Unique Words: 227
The Number of Sentences are: 137861 


## Implementing Pre-Processing

In [6]:
def text_to_ids(source_text,target_text,source_vocab_to_int,target_vocab_to_int):
    source_text_id=[]
    target_text_id=[]
    
    id_eos=target_vocab_to_int['<EOS>']
    
    for source_sentence in source_text.split('\n'):
        a=[source_vocab_to_int[word] for word in source_sentence.split()]
        source_text_id.append(a)
        
    for target_sentence in target_text.split('\n'):
        b=[target_vocab_to_int[word] for word in target_sentence.split()]
        b.append(id_eos)
        target_text_id.append(b)
    
    return source_text_id,target_text_id
        
    

### Pre-Processing Data and Saving it


In [8]:
helper.preprocess_and_save_data(source_path, target_path, text_to_ids)

### CheckPoint
If you ever decide to come back to this notebook or have to restart the notebook, you can start from here. The preprocessed data has been saved to disk.

In [9]:
(source_int_text, target_int_text), (source_vocab_to_int, target_vocab_to_int), _ = helper.load_preprocess()

## Builiding Neural Network

### Input


In [10]:
def model_input():
    input_=tf.placeholder(tf.int32,[None,None],name='input')
    targets=tf.placeholder(tf.int32,[None,None])
    learning_rate=tf.placeholder(tf.float32)
    keep_prob=tf.placeholder(tf.float32,name='keep_prob')
    
    return input_,targets,learning_rate,keep_prob

### Processing Decoding Input

In [ ]:
def process_decoding_input(target_data,target_vocab_to_int,batch_size):
    